In this lab, we will be cleaning data and preparing it for Customer segmentation using k-means clustering.
<br>Please note that this project requires the Olist dataset in <a href='https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce/data?select=olist_customers_dataset.csv'>This link</a>

In [86]:
# load the library

import pandas as pd
import numpy as np

#place the location in the path var
path='C:\\Users\\Wael7\\Data Analytics Course\\Data\\orderItems.csv'
#loading the file on df
df = pd.read_csv(path)
#dropping order_item_id as its only use is displaying number of items ordered (unique and same order)
df=df.drop(['seller_id','shipping_limit_date','order_item_id'],axis=1)
df.head(10)



,order_id,product_id,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,199.90,18.14
5,00048cc3ae777c65dbb7d2a0634bc1ea,ef92defde845ab8450f9d70c526ef70f,21.90,12.69
6,00054e8431b9d7675808bcb819fb4a32,8d4f2bb7e93e6710a28f34fa83ee7d28,19.90,11.85
7,000576fe39319847cbb9d288c5617fa6,557d850972a7d6f792fd18ae1400d9b6,810.00,70.75
8,0005a1a1728c9d785b8e2b08b904576c,310ae3c140ff94b03219ad0adc3c778f,145.95,11.65
9,0005f50442cb953dcd1d21e1fb923495,4535b0e1091c278dfd193e5a1d63b39f,53.99,11.40


For now we will only need Unique order_id because it only shows how many products are requested in a single order

In [87]:
df=df.drop_duplicates(subset=['order_id'])
df[df.duplicated(['order_id'], keep=False)]

,order_id,product_id,price,freight_value


Only unique orders are kept.
Loading the order Dataset to merge the orders with their customers.
We will remove the order timestamps for now.

In [88]:
path='C:\\Users\\Wael7\\Data Analytics Course\\Data\\orders.csv'
orderDf = pd.read_csv(path)#using this to remove duplicates of the same produuct
orderDf = orderDf.loc[:, ['customer_id','order_id']]# removing all columns except order_id and product_id
orderDf


,customer_id,order_id
0,9ef432eb6251297304e76186b10a928d,e481f51cbdc54678b7cc49136f2d6af7
1,b0830fb4747a6c6d20dea0b8c802d7ef,53cdb2fc8bc7dce0b6741e2150273451
2,41ce2a54c0b03bf3443c3d931a367089,47770eb9100c2d0c44946d9cf07ec65d
3,f88197465ea7920adcdbec7375364d82,949d5b44dbf5de918fe9c16f97b45f8a
4,8ab97904e6daea8866dbdbc4fb7aad2c,ad21c59c0840e6cb83a9ceb5573f8159
...,...,...
99436,39bd1228ee8140590ac3aca26f2dfe00,9c5dedf39a927c1b2549525ed64a053c
99437,1fca14ff2861355f6e5f14306ff977a7,63943bddc261676b46f01ca7ac2f7bd8
99438,1aa71eb042121263aafbe80c1b562c9c,83c1379a015df1e13d02aae0204711ab
99439,b331b74b18dc79bcdf6532d51e1637c1,11c177c8e97725db2631073c19f07b62


In [89]:
#merging both dataframes order and review
df = pd.merge(df, orderDf, on='order_id')
df.head(10)

,order_id,product_id,price,freight_value,customer_id
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,58.90,13.29,3ce436f183e68e07877b285a838db11a
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795
5,00048cc3ae777c65dbb7d2a0634bc1ea,ef92defde845ab8450f9d70c526ef70f,21.90,12.69,816cbea969fe5b689b39cfc97a506742
6,00054e8431b9d7675808bcb819fb4a32,8d4f2bb7e93e6710a28f34fa83ee7d28,19.90,11.85,32e2e6ab09e778d99bf2e0ecd4898718
7,000576fe39319847cbb9d288c5617fa6,557d850972a7d6f792fd18ae1400d9b6,810.00,70.75,9ed5e522dd9dd85b4af4a077526d8117
8,0005a1a1728c9d785b8e2b08b904576c,310ae3c140ff94b03219ad0adc3c778f,145.95,11.65,16150771dfd4776261284213b89c304e
9,0005f50442cb953dcd1d21e1fb923495,4535b0e1091c278dfd193e5a1d63b39f,53.99,11.40,351d3cb2cee3c7fd0af6616c82df21d3


Now we will be merging our dataframe with the customers dataset

In [90]:
#merging dataframe with product dataframe for category var
path='C:\\Users\\Wael7\\Data Analytics Course\\Data\\customers.csv'
customerDf = pd.read_csv(path)

merged_df = pd.merge(df, customerDf, on='customer_id', suffixes=('', '_customer'))
merged_df.head(10)
# df.head(1)


# Save cleaned dataframe to CSV file ''

,order_id,product_id,price,freight_value,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,58.90,13.29,3ce436f183e68e07877b285a838db11a,871766c5855e863f6eccc05f988b23cb,28013,campos dos goytacazes,RJ
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,eb28e67c4c0b83846050ddfb8a35d051,15775,santa fe do sul,SP
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,3818d81c6709e39d06b2738a8d3a2474,35661,para de minas,MG
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,af861d436cfc08b2c2ddefd0ba074622,12952,atibaia,SP
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,64b576fb70d441e8f1b2d7d446e483c5,13226,varzea paulista,SP
5,00048cc3ae777c65dbb7d2a0634bc1ea,ef92defde845ab8450f9d70c526ef70f,21.90,12.69,816cbea969fe5b689b39cfc97a506742,85c835d128beae5b4ce8602c491bf385,38017,uberaba,MG
6,00054e8431b9d7675808bcb819fb4a32,8d4f2bb7e93e6710a28f34fa83ee7d28,19.90,11.85,32e2e6ab09e778d99bf2e0ecd4898718,635d9ac1680f03288e72ada3a1035803,16700,guararapes,SP
7,000576fe39319847cbb9d288c5617fa6,557d850972a7d6f792fd18ae1400d9b6,810.00,70.75,9ed5e522dd9dd85b4af4a077526d8117,fda4476abb6307ab3c415b7e6d026526,11702,praia grande,SP
8,0005a1a1728c9d785b8e2b08b904576c,310ae3c140ff94b03219ad0adc3c778f,145.95,11.65,16150771dfd4776261284213b89c304e,639d23421f5517f69d0c3d6e6564cf0e,11075,santos,SP
9,0005f50442cb953dcd1d21e1fb923495,4535b0e1091c278dfd193e5a1d63b39f,53.99,11.40,351d3cb2cee3c7fd0af6616c82df21d3,0782c41380992a5a533489063df0eef6,6636,jandira,SP


We have successfully created our working dataframe of purchases users have made after removing redundant features it will be help us into classifying users. we will be changing the state names for easier data reading later

In [91]:
state_mapping_brazil = {
    'AC': 'Acre', 'AL': 'Alagoas', 'AP': 'Amapá', 'AM': 'Amazonas', 'BA': 'Bahia',
    'CE': 'Ceará', 'DF': 'Distrito Federal', 'ES': 'Espírito Santo', 'GO': 'Goiás', 'MA': 'Maranhão',
    'MT': 'Mato Grosso', 'MS': 'Mato Grosso do Sul', 'MG': 'Minas Gerais', 'PA': 'Pará', 'PB': 'Paraíba',
    'PR': 'Paraná', 'PE': 'Pernambuco', 'PI': 'Piauí', 'RJ': 'Rio de Janeiro', 'RN': 'Rio Grande do Norte',
    'RS': 'Rio Grande do Sul', 'RO': 'Rondônia', 'RR': 'Roraima', 'SC': 'Santa Catarina',
    'SP': 'São Paulo', 'SE': 'Sergipe', 'TO': 'Tocantins'
}
merged_df['customer_state'] = merged_df['customer_state'].replace(state_mapping_brazil)
merged_df.to_csv('CleanDF.csv',index=False)

Another way of understanding our customers needs to determine the categories of most bought category of items in a state, we will have to create another dataframe for efficiency

In [92]:
merged_df=merged_df[['product_id','customer_state']]
merged_df.head(10)

,product_id,customer_state
0,4244733e06e7ecb4970a6e2683c13e61,Rio de Janeiro
1,e5f2d52b802189ee658865ca93d83a8f,São Paulo
2,c777355d18b72b67abbeef9df44fd0fd,Minas Gerais
3,7634da152a4610f1595efa32f14722fc,São Paulo
4,ac6c3623068f30de03045865e4e10089,São Paulo
5,ef92defde845ab8450f9d70c526ef70f,Minas Gerais
6,8d4f2bb7e93e6710a28f34fa83ee7d28,São Paulo
7,557d850972a7d6f792fd18ae1400d9b6,São Paulo
8,310ae3c140ff94b03219ad0adc3c778f,São Paulo
9,4535b0e1091c278dfd193e5a1d63b39f,São Paulo


In [93]:
df = pd.read_csv("C:\\Users\\Wael7\\Data Analytics Course\\Data\\products.csv")
df=df[['product_id','product_category_name']]
merged_df=pd.merge(df, merged_df, on='product_id')
merged_df.head(10)

,product_id,product_category_name,customer_state
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,São Paulo
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,Rio de Janeiro
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,São Paulo
3,cef67bcfe19066a932b7673e239eb23d,bebes,São Paulo
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,São Paulo
5,41d3672d4792049fa1779bb35283ed13,instrumentos_musicais,Minas Gerais
6,732bd381ad09e530fe0a5f457d81becb,cool_stuff,São Paulo
7,732bd381ad09e530fe0a5f457d81becb,cool_stuff,Rio de Janeiro
8,2548af3e6e77a690cf3eb6368e9ab61e,moveis_decoracao,São Paulo
9,2548af3e6e77a690cf3eb6368e9ab61e,moveis_decoracao,Minas Gerais


In [94]:
df = pd.read_csv("C:\\Users\\Wael7\\Data Analytics Course\\Data\\productCategoryName.csv")
merged_df = pd.merge(merged_df,df, on='product_category_name', how='left')
merged_df.to_csv('productDF.csv',index=False)

In [96]:
merged_df.head(10)

,product_id,product_category_name,customer_state,product_category_name_english
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,São Paulo,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,Rio de Janeiro,art
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,São Paulo,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,bebes,São Paulo,baby
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,São Paulo,housewares
5,41d3672d4792049fa1779bb35283ed13,instrumentos_musicais,Minas Gerais,musical_instruments
6,732bd381ad09e530fe0a5f457d81becb,cool_stuff,São Paulo,cool_stuff
7,732bd381ad09e530fe0a5f457d81becb,cool_stuff,Rio de Janeiro,cool_stuff
8,2548af3e6e77a690cf3eb6368e9ab61e,moveis_decoracao,São Paulo,furniture_decor
9,2548af3e6e77a690cf3eb6368e9ab61e,moveis_decoracao,Minas Gerais,furniture_decor
